In [170]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from scipy import stats
import chart_studio
import chart_studio.plotly as py
from stats_can import StatsCan
sc = StatsCan()

#chartstudio
username = 'somayaji' # your username
api_key = 'SIzBjYDtWqWAw4sf2PNr' # your api key - go to profile > settings > regenerate keychart_studio.tools.set_credentials_file(username=username, api_key=api_key)
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

#df = sc.table_to_df("271-000-22-01")
#df = sc.vectors_to_df(["v74804", "v41692457"])
df = sc.table_to_df("131-007-66-01")

pd.set_option("display.max_rows", None, "display.max_columns", None)
df.head()


,REF_DATE,GEO,DGUID,Case identifier number,Case information,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2020-01-01,Canada,NaN,1,Date case was last updated - month,Number,223,units,0,v1146068950,1.1.1,4.0,NaN,NaN,NaN,0
1,2020-01-01,Canada,NaN,1,Date case was last updated - day,Number,223,units,0,v1146068951,1.1.2,16.0,NaN,NaN,NaN,0
2,2020-01-01,Canada,NaN,1,Episode date - month,Number,223,units,0,v1146068952,1.1.3,1.0,NaN,NaN,NaN,0
3,2020-01-01,Canada,NaN,1,Episode date - day,Number,223,units,0,v1146068953,1.1.4,21.0,NaN,NaN,NaN,0
4,2020-01-01,Canada,NaN,1,Gender,Number,223,units,0,v1146068954,1.1.5,1.0,NaN,NaN,NaN,0


In [171]:
to_drop = ['GEO', 
 'DGUID', 
 'UOM', 
 'SCALAR_FACTOR', 
 'STATUS', 
 'SYMBOL', 
 'TERMINATED', 
 'DECIMALS']

df.drop(columns=to_drop, inplace=True)

In [172]:
df.head()

,REF_DATE,Case identifier number,Case information,UOM_ID,SCALAR_ID,VECTOR,COORDINATE,VALUE
0,2020-01-01,1,Date case was last updated - month,223,0,v1146068950,1.1.1,4.0
1,2020-01-01,1,Date case was last updated - day,223,0,v1146068951,1.1.2,16.0
2,2020-01-01,1,Episode date - month,223,0,v1146068952,1.1.3,1.0
3,2020-01-01,1,Episode date - day,223,0,v1146068953,1.1.4,21.0
4,2020-01-01,1,Gender,223,0,v1146068954,1.1.5,1.0


In [173]:
df2 = df.pivot(index='Case identifier number', columns='Case information', values='VALUE')

In [174]:
df2.head()

Case information,Age group,Date case was last updated - day,Date case was last updated - month,Death,"Death, previous status",Episode date - day,Episode date - month,Gender,Hospitalization,"Hospitalization, previous status",Intensive care unit,"Intensive care unit, previous status",Transmission
Case identifier number,,,,,,,,,,,,,
1,5.0,16.0,4.0,2.0,2.0,21.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0
10,4.0,14.0,4.0,2.0,2.0,18.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0
100,4.0,7.0,4.0,2.0,2.0,10.0,3.0,1.0,2.0,2.0,2.0,2.0,1.0
1000,5.0,9.0,4.0,2.0,2.0,20.0,3.0,1.0,1.0,1.0,2.0,2.0,1.0
10000,99.0,14.0,4.0,2.0,2.0,8.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0


In [175]:
#removing unanswered age group rows for display
age_df = df2.drop(df2[(df2['Age group'] == 99.0)].index)
age_df['Age group'].replace({1: "0 to 19", 2:"20 to 29", 3:"30 to 39", 4:"40 to 49", 5:"50 to 59", 6:"60 to 69", 7:"70 to 79", 8:"80+" }, inplace=True)

age_df_m = age_df.drop(age_df[(age_df['Gender'] == 1.0)].index) # male
age_df_f = age_df.drop(age_df[(age_df['Gender'] == 2.0)].index) # female
age_df_u = age_df.drop(age_df[(age_df['Gender'] == 7.0)].index) # unknown

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=age_df_m['Age group'],
    name = 'male'
))

fig.add_trace(go.Histogram(
    x=age_df_f['Age group'],
    name = 'female'
))

fig.add_trace(go.Histogram(
    x=age_df_u['Age group'],
    name = 'unknown'
))

fig.update_layout(
    xaxis_type='category',
    xaxis_categoryorder='category ascending',
    title_text='Age Distribution of Canadian COVID-19 Cases', # title of plot
    xaxis_title_text='Age Group', # xaxis label
    yaxis_title_text='Count', # yaxis label
)

fig.update_traces(opacity=0.75)
#fig.to_json()
fig.show()

In [176]:
# sunburst
df_sun = df2.drop(df2[(df2['Age group'] == 99.0)].index)
df_sun['count'] = 1
df_sun['Age group'].replace({1: "0 to 19", 2:"20 to 29", 3:"30 to 39", 4:"40 to 49", 5:"50 to 59", 6:"60 to 69", 7:"70 to 79", 8:"80+" }, inplace=True)
df_sun['Gender'].replace({1:"Male",2:"Female",3:"Unknown"}, inplace=True)
df_sun['Death'].replace({1:"Yes",2:"No",9:"Not Stated"}, inplace=True)
fig = px.sunburst(df_sun, path=['Gender','Age group', 'Death'], values='count')
fig.show()
#fig.to_json()
#py.plot(fig, filename = 'Death_Sunburst', auto_open=True)

#sex: 1 = male, 2 = female, 3= unknown
#age group: 1


In [203]:
#drilling into the data above, we can pull a few statistics out:
df_MaleSubset = df_sun[(df_sun["Gender"] == "Male")]
TotalMaleCount = df_MaleSubset.count()
df_MaleSubset_death= df_MaleSubset[(df_MaleSubset["Death"] == "Yes")]
MaleDeathCount = df_MaleSubset_death.count()

print("Total Number of Males: " + str(TotalMaleCount['Age group']))
print("Total Number of Male Deaths: " + str(MaleDeathCount['Age group']))
print("Ratio of Male Deaths: " + str(MaleDeathCount['Age group']/TotalMaleCount['Age group']))

df_FemaleSubset = df_sun[(df_sun["Gender"] == "Female")]
TotalFemaleCount = df_FemaleSubset.count()
df_FemaleSubset_death= df_FemaleSubset[(df_FemaleSubset["Death"] == "Yes")]
FemaleDeathCount = df_FemaleSubset_death.count()

print("Total Number of Females: " + str(TotalFemaleCount['Age group']))
print("Total Number of Female Deaths: " + str(FemaleDeathCount['Age group']))
print("Ratio of Female Deaths: " + str(FemaleDeathCount['Age group']/TotalFemaleCount['Age group']))


Total Number of Males: 7346
Total Number of Male Deaths: 307
Ratio of Male Deaths: 0.0417914511298666
Total Number of Females: 9111
Total Number of Female Deaths: 283
Ratio of Female Deaths: 0.031061354406761058


In [239]:
#checking if our data is normally distributed
df_MaleSubsetNormalized = df_MaleSubset['Death'].replace({"Yes":1,"No":0,"Not Stated":9})
df_MaleSubsetNormalized = df_MaleSubsetNormalized[df_MaleSubsetNormalized != 9]
MaleArray = df_MaleSubsetNormalized.values

df_FemaleSubsetNormalized = df_FemaleSubset['Death'].replace({"Yes":1,"No":0,"Not Stated":9})
df_FemaleSubsetNormalized = df_FemaleSubsetNormalized[df_FemaleSubsetNormalized != 9]
FemaleArray = df_FemaleSubsetNormalized.values

stat, p = stats.ttest_ind(MaleArray, FemaleArray, equal_var=False)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=2.382, p=0.017
Probably different distributions
